# process_json

Import json from the revisions folder.

Example format:

[
  {
    "geometry": {
      "coordinates": [
        "-86.657509",
        "36.257828",
        0
      ],
      "type": "Point"
    },
    "properties": {
      "is_broken": false,
      "is_active": true,
      "state": "TN",
      "city": "Old Hickory",
      "street": "1252 Robinson Rd",
      "last_checked": "Checked 30 minutes ago"
    },
    "type": "Feature"
  },
  {
    "geometry": {
      "coordinates": [
        "-87.600054",
        "37.986435",
        0
      ],
      "type": "Point"
    },
    "properties": {
      "is_broken": false,
      "is_active": false,
      "state": "IN",
      "city": "Evansville",
      "street": "909 N St Joseph Ave",
      "last_checked": "Checked 30 minutes ago"
    },
    "type": "Feature"
  },

...

In [29]:
# Packages
import os
import json
import pandas as pd

In [30]:
# Get files in revisions directory
revisions = os.listdir('revisions')
revisions

['0001.Archive_from_Sat_Feb_15_190114_UTC_2025.dd9d13020edc47bd1bd76b01413341d7d98a2ea1.mcbroken.json',
 '0002.Archive_from_Fri_Feb_14_193109_UTC_2025.7746fc991b21f3cee759e8f06c3fe146ef453823.mcbroken.json',
 '0003.Archive_from_Fri_Feb_14_190113_UTC_2025.9f8315b80e5d46067bb32439765335b26d5f8b82.mcbroken.json',
 '0004.Archive_from_Thu_Feb_13_193107_UTC_2025.f0be995e3019f047e859f5ee38bb10baef503b05.mcbroken.json',
 '0005.Archive_from_Thu_Feb_13_190115_UTC_2025.7737c110a5e9cb65f7f115e5303e2d4c80d1db4f.mcbroken.json',
 '0006.Archive_from_Wed_Feb_12_193111_UTC_2025.61c548b5e006dbc89296e26555dc95a2e7520c58.mcbroken.json',
 '0007.Archive_from_Wed_Feb_12_190122_UTC_2025.af34ba322981e01cde925ed2d9898f95c751d90f.mcbroken.json',
 '0008.Archive_from_Tue_Feb_11_193100_UTC_2025.34707f3f22102278ba70f648865c32ca70acc4cd.mcbroken.json',
 '0009.Archive_from_Tue_Feb_11_190121_UTC_2025.8ec8368cf5b01ae39351ff032c83badd2ed3db87.mcbroken.json',
 '0010.Archive_from_Mon_Feb_10_193105_UTC_2025.f5a024722c8e334f1

In [31]:
# Load first json as example
js = json.load(open('revisions/' + revisions[0]))
js

[{'geometry': {'coordinates': ['-73.988281', '40.71883', 0.0],
   'type': 'Point'},
  'properties': {'is_broken': False,
   'is_active': True,
   'dot': 'working',
   'state': 'NY',
   'city': 'New York',
   'street': '114 Delancey St',
   'country': 'USA',
   'last_checked': 'Checked 146 minutes ago'},
  'type': 'Feature'},
 {'geometry': {'coordinates': ['-74.00509', '40.728794', 0.0],
   'type': 'Point'},
  'properties': {'is_broken': True,
   'is_active': True,
   'dot': 'broken',
   'state': 'NY',
   'city': 'New York',
   'street': '208 Varick St',
   'country': 'USA',
   'last_checked': 'Checked 147 minutes ago'},
  'type': 'Feature'},
 {'geometry': {'coordinates': ['-73.993408', '40.729197', 0.0],
   'type': 'Point'},
  'properties': {'is_broken': False,
   'is_active': True,
   'dot': 'working',
   'state': 'NY',
   'city': 'New York',
   'street': '724 Broadway',
   'country': 'USA',
   'last_checked': 'Checked 146 minutes ago'},
  'type': 'Feature'},
 {'geometry': {'coordinat

## Function to Process

In [32]:
# Function to create dataframe with relevant info

def json_to_df(json_file):

    print(json_file)

    # Open file
    js = json.load(open('revisions/' + json_file))

    # Cleaned up string
    # Consolidate multiple "__" into one
    cleaned_str = json_file.replace('__', '_')

    # Parse datetime
    # Example file name: '0977.Archive_from_Wed_Sep_21_190538_UTC_2022.0adcd7c63d6a808c7b1cf0d1fe49f23ef1524498.mcbroken.json'
    year = int(cleaned_str.split('_')[-1].split('.')[0])
    mon_string = cleaned_str.split('_')[3]
    mon = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'].index(mon_string) + 1
    day = int(cleaned_str.split('_')[4])
    hr = int(cleaned_str.split('_')[5][:2])
    min = int(cleaned_str.split('_')[5][2:4])
    sec = int(cleaned_str.split('_')[5][4:6])
    datetime = pd.to_datetime(f'{year}-{mon}-{day} {hr}:{min}:{sec}')
    # Also just the date part
    date = pd.to_datetime(f'{year}-{mon}-{day}')

    # Initialize counters
    total_machines = 0
    broken_machines = 0

    # Loop through the data and count
    for entry in js:
        total_machines += 1
        if entry['properties']['is_broken']:
            broken_machines += 1

    # Output dataframe: date, datetime, total_machines, broken_machines
    df = pd.DataFrame({'date': [date],
                       'datetime': [datetime],
                       'total_machines': [total_machines],
                       'broken_machines': [broken_machines]})
    return df
    

In [33]:
# Test on first file
print(revisions[0])
json_to_df(revisions[0])

0001.Archive_from_Sat_Feb_15_190114_UTC_2025.dd9d13020edc47bd1bd76b01413341d7d98a2ea1.mcbroken.json
0001.Archive_from_Sat_Feb_15_190114_UTC_2025.dd9d13020edc47bd1bd76b01413341d7d98a2ea1.mcbroken.json


,date,datetime,total_machines,broken_machines
0,2025-02-15,2025-02-15 19:01:14,13159,1267


## Run for all files

In [34]:
# Run function for all files
dfs = []
for revision in revisions:
    dfs.append(json_to_df(revision))

# Concatenate all dataframes
df = pd.concat(dfs, ignore_index=True)
df

0001.Archive_from_Sat_Feb_15_190114_UTC_2025.dd9d13020edc47bd1bd76b01413341d7d98a2ea1.mcbroken.json
0002.Archive_from_Fri_Feb_14_193109_UTC_2025.7746fc991b21f3cee759e8f06c3fe146ef453823.mcbroken.json
0003.Archive_from_Fri_Feb_14_190113_UTC_2025.9f8315b80e5d46067bb32439765335b26d5f8b82.mcbroken.json
0004.Archive_from_Thu_Feb_13_193107_UTC_2025.f0be995e3019f047e859f5ee38bb10baef503b05.mcbroken.json
0005.Archive_from_Thu_Feb_13_190115_UTC_2025.7737c110a5e9cb65f7f115e5303e2d4c80d1db4f.mcbroken.json
0006.Archive_from_Wed_Feb_12_193111_UTC_2025.61c548b5e006dbc89296e26555dc95a2e7520c58.mcbroken.json
0007.Archive_from_Wed_Feb_12_190122_UTC_2025.af34ba322981e01cde925ed2d9898f95c751d90f.mcbroken.json
0008.Archive_from_Tue_Feb_11_193100_UTC_2025.34707f3f22102278ba70f648865c32ca70acc4cd.mcbroken.json
0009.Archive_from_Tue_Feb_11_190121_UTC_2025.8ec8368cf5b01ae39351ff032c83badd2ed3db87.mcbroken.json
0010.Archive_from_Mon_Feb_10_193105_UTC_2025.f5a024722c8e334f1502d905330ff160f8504c94.mcbroken.json


,date,datetime,total_machines,broken_machines
0,2025-02-15,2025-02-15 19:01:14,13159,1267
1,2025-02-14,2025-02-14 19:31:09,11428,1068
2,2025-02-14,2025-02-14 19:01:13,13185,1308
3,2025-02-13,2025-02-13 19:31:07,11707,1189
4,2025-02-13,2025-02-13 19:01:15,13217,1356
...,...,...,...,...
1709,2020-10-27,2020-10-27 19:23:43,6886,351
1710,2020-10-26,2020-10-26 19:40:29,6844,373
1711,2020-10-26,2020-10-26 19:23:41,6377,352
1712,2020-10-25,2020-10-25 19:40:08,6818,354


## Keep most recent item for each day

In [37]:
# On each date, keep item with greatest datetime
max_dts = df.groupby('date')['datetime'].max()
df = df[df['datetime'].isin(max_dts)]

## Output to CSV

In [38]:
# Output to CSV

# Get most recent date
most_recent_date = df['date'].max()

# Put most recent date in filename
df.to_csv('mcbroken_daily_most_recent_on_' + most_recent_date.strftime('%Y%m%d') + '.csv', index=False)